# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
df = pd.read_csv("../WeatherPy/output_data/cities.csv")
df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Busselton,-33.6500,115.3333,72.09,36,6,4.76,AU,1638414576
1,1,Port Alfred,-33.5906,26.8910,67.46,92,93,3.11,ZA,1638414576
2,2,Tahe,52.3333,124.7333,7.74,78,100,7.61,CN,1638414576
3,3,Qaanaaq,77.4840,-69.3632,5.56,68,12,6.89,GL,1638414577
4,4,Punta Arenas,-53.1500,-70.9167,44.71,70,20,9.22,CL,1638414389


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
gmaps.configure(api_key=g_key)

In [9]:
df = df.dropna()
locations = df[["Latitude", "Longitude"]].astype(float)
humidity = df["Humidity"].astype(float)

In [10]:
fig = gmaps.figure(center=(20,0),zoom_level=1.7)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=350,
                                 point_radius = 3)

fig.add_layer(heat_layer)


In [11]:
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
perfectdf = df[(df["Max Temperature"] < 70) & (df["Max Temperature"] > 50) & (df["Humidity"] < 40) & (df["Cloudiness"] <20)]
perfectdf.dropna()

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,Katsuura,35.1333,140.3000,53.71,35,0,4.79,JP,1638414578
89,89,Hasaki,35.7333,140.8333,57.29,34,0,15.84,JP,1638414597
182,182,Khuzdar,27.7384,66.6434,52.84,39,0,1.59,PK,1638414623
202,202,Comodoro Rivadavia,-45.8667,-67.5000,55.26,30,0,27.63,AR,1638414629
277,277,Marzuq,14.4000,46.4667,54.63,35,0,0.89,YE,1638414649
329,329,Yining,29.0361,114.5636,54.88,34,9,1.83,CN,1638414662
380,380,Portales,34.1862,-103.3344,51.94,38,5,4.92,US,1638414540
384,384,Saint George,37.1041,-113.5841,56.53,32,1,1.99,US,1638414590
396,396,Najrān,17.4924,44.1277,52.21,34,0,3.40,SA,1638414547
437,437,Shaoyang,27.2315,111.4525,53.94,33,0,1.72,CN,1638414691


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_df = perfectdf[["City", "Country", "Latitude", "Longitude"]]
hotel_df["Hotel Name"] = ""
hotel_df

<ipython-input-13-52ac8b172d2b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Latitude,Longitude,Hotel Name
10,Katsuura,JP,35.1333,140.3000,
89,Hasaki,JP,35.7333,140.8333,
182,Khuzdar,PK,27.7384,66.6434,
202,Comodoro Rivadavia,AR,-45.8667,-67.5000,
277,Marzuq,YE,14.4000,46.4667,
329,Yining,CN,29.0361,114.5636,
380,Portales,US,34.1862,-103.3344,
384,Saint George,US,37.1041,-113.5841,
396,Najrān,SA,17.4924,44.1277,
437,Shaoyang,CN,27.2315,111.4525,


In [14]:
for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{row.Latitude}, {row.Longitude}", 
        "radius": "5000",
        "type": "lodging",
        "key": g_key,
    }
    print(f"Retrieving Results for Index {index}: {row.City}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Retrieving Results for Index 10: Katsuura.
Closest hotel is Katsuura Hotel Mikazuki.
------------
Retrieving Results for Index 89: Hasaki.


C:\Users\maryclarecc\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Closest hotel is Hotel Sunrise Choshi.
------------
Retrieving Results for Index 182: Khuzdar.
Closest hotel is jhalwan Rent A Car Moola Chotok Boking Office.
------------
Retrieving Results for Index 202: Comodoro Rivadavia.
Closest hotel is Austral Hotel.
------------
Retrieving Results for Index 277: Marzuq.
Missing field/result... skipping.
------------
Retrieving Results for Index 329: Yining.
Closest hotel is Xiushui Hotel.
------------
Retrieving Results for Index 380: Portales.
Closest hotel is Holiday Inn Express & Suites Portales, an IHG Hotel.
------------
Retrieving Results for Index 384: Saint George.
Closest hotel is Best Western Plus Abbey Inn.
------------
Retrieving Results for Index 396: Najrān.
Closest hotel is محمد هادي ال فطيح.
------------
Retrieving Results for Index 437: Shaoyang.
Closest hotel is IU Hotel Shaoyang Xihu Road.
------------
Retrieving Results for Index 451: Riyadh.
Closest hotel is InterContinental Riyadh, an IHG Hotel.
------------
Retrieving Res

In [15]:
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
10,Katsuura,JP,35.1333,140.3000,Katsuura Hotel Mikazuki
89,Hasaki,JP,35.7333,140.8333,Hotel Sunrise Choshi
182,Khuzdar,PK,27.7384,66.6434,jhalwan Rent A Car Moola Chotok Boking Office
202,Comodoro Rivadavia,AR,-45.8667,-67.5000,Austral Hotel
277,Marzuq,YE,14.4000,46.4667,
329,Yining,CN,29.0361,114.5636,Xiushui Hotel
380,Portales,US,34.1862,-103.3344,"Holiday Inn Express & Suites Portales, an IHG ..."
384,Saint George,US,37.1041,-113.5841,Best Western Plus Abbey Inn
396,Najrān,SA,17.4924,44.1277,محمد هادي ال فطيح
437,Shaoyang,CN,27.2315,111.4525,IU Hotel Shaoyang Xihu Road


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [17]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
hotel_layer = gmaps.symbol_layer(
    locations,
    info_box_content=hotel_info
)
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))